In [1]:
import matplotlib.pyplot as plt
import os, re
import pandas as pd
import numpy as np

from datetime import datetime as dt
import cPickle as pickle
import re #Regex
import seaborn as sns
sns.set()
%matplotlib inline

# 1. Evaluating Crawling Indicator

#### Terlihat total penggunaan waktu dan memory sangat signifikan, hal tersebut karena jumlah berita positif dan negatif yang berbeda jauh

In [8]:
# %time DATACrawl = pickle.load( open( "v1.4\DATA_ANALSISIS_crawl.p", "rb" ) )
%time DATACrawl = pickle.load( open( "v1.3\df_mysql.p", "rb" ) )
# %time DATACrawl = pickle.load( open( "v1.4\DATA_CRAWL-v4b.p", "rb" ) )
Data_Analisis = DATACrawl[DATACrawl.flag.isin([-1,1])][~DATACrawl.crawlMem.isnull()]

crawlIndicator = pd.DataFrame([Data_Analisis.crawlMem,Data_Analisis.crawlTime]).transpose()
feedIndicator  = pd.DataFrame([Data_Analisis.feedMem,Data_Analisis.feedTime]).transpose()
print crawlIndicator.sum()
print feedIndicator.sum()

Wall time: 6.7 s
crawlMem     4442320.0000
crawlTime        753.4597
dtype: float64
feedMem     5655162.00000
feedTime         32.16013
dtype: float64


# 2.Evaluating the Classifier v1.4 vs v1.6 vs v2.0


## 2.1 Load from the 'Pickled' dump

In [10]:
%time DATACrawl = pickle.load( open( "v1.4\DATA_CRAWL-v4b.p", "rb" ) )
Data_Analisis = DATACrawl[DATACrawl.flag.isin([-1,1])][~DATACrawl.crawlMem.isnull()]
DATAStd  = pd.DataFrame()

#Load standard Text
DATAStd['Judul'] = Data_Analisis.judul
DATAStd['Short'] = Data_Analisis.shortDesc
DATAStd['Long']  = Data_Analisis.longDesc

# DATAStd['CrawlMem']   = Data_Analisis.crawlMem
# DATAStd['CrawlTime']  = Data_Analisis.crawlTime
# DATAStd['FeedMem']    = Data_Analisis.feedMem
# DATAStd['FeedTime']   = Data_Analisis.feedTime

%time DATA_TARGET = Data_Analisis.flag

DATA_TARGET = DATA_TARGET.reset_index(drop=True)
DATAStd     = DATAStd.reset_index(drop=True)

Wall time: 65 ms
Wall time: 0 ns


In [11]:
#We only use 2% of total negative instance
factor = 100.0/100
size   = int(factor*DATA_TARGET[DATA_TARGET.values == -1].count())
print 'Total Negative Instances    : ',DATA_TARGET[DATA_TARGET.values == -1].count()
print 'Training Negative Instances : ',size
print 'Training Positif Instances  : ',DATA_TARGET[DATA_TARGET.values == 1].count()

Total Negative Instances    :  324
Training Negative Instances :  324
Training Positif Instances  :  146


## 2. Evaluate

### Preprocessing

![smiley](img/evaluasi preproc2.png)

In [12]:
from HTMLParser import HTMLParser
import string
import re #Regex
def preprocK1(par):
    
    def onlyAZ(teks):
        return re.sub(r'[^a-zA-Z]',' ', teks)
    
    class MLStripper(HTMLParser):
        def __init__(self):
            self.reset()
            self.fed = []
        def handle_data(self, d):
            self.fed.append(d)
        def get_data(self):
            return ' '.join(self.fed)

    def strip_tags(html):
        s = MLStripper()
        s.feed(html)
        striped  = s.get_data()                         #get HTML-Tags free text
        lowers   = striped.lower()                      #Lowercase all words
        nospace  = lowers.strip()                       #Remove leading and trailing white space
        return nospace
    
    par         = strip_tags(par)
    onlyAZs     = onlyAZ(par)
    return onlyAZs

In [13]:
d = DATAStd['Short'][0]
print d
print preprocK1(d)

<img src="http://cdn-media.viva.id/thumbs2/2014/11/25/281549_pekerja-perkebunan-kelapa-sawit-di-sumatra-utara_663_382.jpg" align="left" hspace="7" width="100" />Laba anjlok sampai 75 persen.
laba anjlok sampai    persen 


In [14]:
from HTMLParser import HTMLParser
import string
import re #Regex

#Stopwords data source : https://www.illc.uva.nl/Research/Publications/Reports/MoL-2003-02.text.pdf
stopword_html = open("id.stopwords.01.01.2016.txt",'r').read()
stopwords     = stopword_html.split("\n")

def preprocK2(par):

    postagger = pickle.load(open("POSTAGGER.p", "rb"))
    
    def stopwordRemoval(text):
        text = ' '.join([word for word in text.split() if word not in stopwords])
        
        return text
    def onlyAZ(teks):
        return re.sub(r'[^a-zA-Z]',' ', teks)

    def onlyNVFromSentence(teks):
        splitted = postagger.tag(onlyAZ(teks).split())
        nouns = [word for word,pos in splitted \
            if (pos == 'NN' or pos == 'NNP' or 
                pos == 'NNS' or pos == 'VB')]
        nounsSentence = string.join(nouns)
        return nounsSentence

    class MLStripper(HTMLParser):
        def __init__(self):
            self.reset()
            self.fed = []
        def handle_data(self, d):
            self.fed.append(d)
        def get_data(self):
            return ' '.join(self.fed)

    def strip_tags(html):
        s = MLStripper()
        s.feed(html)
        striped  = s.get_data()                         #get HTML-Tags free text
        lowers   = striped.lower()                      #Lowercase all words
        nospace  = lowers.strip()                       #Remove leading and trailing white space
        return nospace

    par         = strip_tags(par)
    par         = stopwordRemoval(par)
    NVPar       = onlyNVFromSentence(par)
    return  NVPar

In [15]:
d = DATAStd['Short'][0]
print d
print preprocK2(d)

<img src="http://cdn-media.viva.id/thumbs2/2014/11/25/281549_pekerja-perkebunan-kelapa-sawit-di-sumatra-utara_663_382.jpg" align="left" hspace="7" width="100" />Laba anjlok sampai 75 persen.
laba anjlok


In [16]:
from HTMLParser import HTMLParser
import string
import re #Regex
import operator
def preprocK3(data):
    
    postagger = pickle.load(open( "POSTAGGER.p", "rb" ))
    tagList   = pickle.load(open( "TAGLIST.p", "rb" ))
    
    def onlyAZ(teks):
        return re.sub(r'[^a-zA-Z]', ' ', teks)
    
    def onlyNVFractionSentence(teks):
        splitted = postagger.tag(onlyAZ(teks).split())
        N  = len(splitted)
        
        word = 0
        k = {}
        for i in tagList:k[i] = float (0)
        
        for word,pos in splitted:
            k[pos]+=1
        return N,k

    def onlyNVFractionParagraph(par):
        splittedPar = par.split('.')
        
        word = 0
        _k = {}
        for i in tagList:_k[i] = float (0)
        
        for sentence in splittedPar:
            w,k = onlyNVFractionSentence(sentence)
            word+=w
            _k = {x: _k.get(x, 0) + k.get(x, 0) for x in set(_k)}
        #return noun/word,verb/word
        return _k.values()
    
    class MLStripper(HTMLParser):
        def __init__(self):
            self.reset()
            self.fed = []
        def handle_data(self, d):
            self.fed.append(d)
        def get_data(self):
            return ''.join(self.fed)

    def strip_tags(html):
        s = MLStripper()
        s.feed(html)
        striped  = s.get_data()                         #get HTML-Tags free text
        lowers   = striped.lower()                      #Lowercase all words
        nospace  = lowers.strip()                       #Remove leading and trailing white space
        return nospace
    
    balikan = np.array([0 for i in range(0,len(tagList))])
    for teks in data:
        frac = onlyNVFractionParagraph(strip_tags(teks))
        balikan = np.vstack((balikan,frac))
    return balikan[1:]

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

dummy1 = DATAStd['Judul']+' '+DATAStd['Short']
dummy2 = DATAStd['Judul']+' '+DATAStd['Long']+' '+DATAStd['Short']

vect = TfidfVectorizer(decode_error='ignore')

%time d1k1 = vect.fit_transform(dummy1.map(lambda x: preprocK1(x)))
%time d1k2 = vect.fit_transform(dummy1.map(lambda x: preprocK2(x)))
%time d1k3 = preprocK3(dummy1)
%time d2k1 = vect.fit_transform(dummy2.map(lambda x: preprocK1(x)))
%time d2k2 = vect.fit_transform(dummy2.map(lambda x: preprocK2(x)))
%time d2k3 = preprocK3(dummy2)

Wall time: 65 ms
Wall time: 11.7 s
Wall time: 162 ms
Wall time: 316 ms
Wall time: 14.8 s
Wall time: 1.06 s


In [32]:
# d1k1
# d1k2

# d2k1
# d2k2


<470x9725 sparse matrix of type '<type 'numpy.float64'>'
	with 46062 stored elements in Compressed Sparse Row format>

In [72]:
print DATAStd.CrawlMem.sum()
print DATAStd.CrawlTime.sum()
print DATAStd.FeedMem.sum()
print DATAStd.FeedTime.sum()

4442320.0
753.4597
5655162
32.1601297744


In [ ]:
# from sklearn import cross_validation
# from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
# from sklearn.metrics import precision_score, recall_score, classification_report, roc_auc_score,roc_curve,auc
# from scipy import interp
# import datetime

# def skorStratCV(clf,X,y,nfolds=10):
#     metrics = []
    
#     trainTime,testTime = [],[]
#     precis,recals,f1s,accs,rocs = [],[],[],[],[]
    
#     mean_tpr = 0.0
#     mean_fpr = np.linspace(0, 1, 100)
#     all_tpr = []
    
#     skf = cross_validation.StratifiedKFold(y, n_folds=nfolds)
#     for train_index, test_index in skf:
#         X_train,y_train = X[train_index],y[train_index]
#         X_test,y_test   = X[test_index],y[test_index]
        
#         #training time
#         start = dt.now()
#         clf.fit(X_train, y_train)
#         end = dt.now()
#         trainTime.append((end-start).microseconds)
        
#         #testing time
#         start = dt.now()
#         yhat = clf.predict(X_test)
#         end = dt.now()
#         testTime.append((end-start).microseconds)
        
#         #score
#         precis.append(precision_score(y_test, yhat, average='binary'))
#         recals.append(recall_score(y_test, yhat, average='binary'))
#         f1s.append(f1_score(y_test, yhat, average='binary'))
#         accs.append(accuracy_score(y_test, yhat))
        
#         probas_              = clf.fit(X_train, y_train).predict_proba(X_test)
#         fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
#         mean_tpr            += interp(mean_fpr, fpr, tpr)
#         mean_tpr[0]          = 0.0
        
#     def ave(lis):
#         return sum(lis)/len(lis)
    
#     metrics.append(ave(trainTime))
#     metrics.append(ave(testTime))

# #     metrics.append(ave(testTime))
#     metrics.append(ave(precis))
#     metrics.append(ave(recals))
#     metrics.append(ave(f1s))
#     metrics.append(ave(accs))
    
#     mean_tpr /= nfolds
#     mean_tpr[-1] = 1.0
#     mean_auc = auc(mean_fpr, mean_tpr)
#     metrics.append(mean_auc)
    
#     return metrics,mean_fpr,mean_tpr

In [ ]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.pipeline import Pipeline

# mnb = MultinomialNB(alpha=0.1)
# svc = SVC(kernel='linear',cache_size=800,probability=True)

# kombinasi  = [{'name' : 'MNB', 'pipe' : mnb},
#               {'name' : 'SVM', 'pipe' : svc},
#              ]

In [ ]:
# kombinasi[0]['pipe']

In [ ]:
# X1 = []
# # X0.append({'name':'Judul','Prep':'Kombinasi1','Data':DATAStd['Judul']})
# # X0.append({'name':'Short','Prep':'Kombinasi1','Data':DATAStd['Short']})
# # X0.append({'name':'Long' ,'Prep':'Kombinasi1','Data':DATAStd['Long']})
# X1.append({'name':'X1' ,'Prep':'K1','Data':d1k1})
# X1.append({'name':'X2' ,'Prep':'K1','Data':d2k1})

# X2 = []
# # X0.append({'name':'Judul','Prep':'Kombinasi1','Data':DATAStd['Judul']})
# # X0.append({'name':'Short','Prep':'Kombinasi1','Data':DATAStd['Short']})
# # X0.append({'name':'Long' ,'Prep':'Kombinasi1','Data':DATAStd['Long']})
# X2.append({'name':'X1' ,'Prep':'K2','Data':d1k2})
# X2.append({'name':'X2' ,'Prep':'K2','Data':d2k2})

# X3 = []
# # X0.append({'name':'Judul','Prep':'Kombinasi1','Data':DATAStd['Judul']})
# # X0.append({'name':'Short','Prep':'Kombinasi1','Data':DATAStd['Short']})
# # X0.append({'name':'Long' ,'Prep':'Kombinasi1','Data':DATAStd['Long']})
# X3.append({'name':'X1' ,'Prep':'K3','Data':d1k3})
# X3.append({'name':'X2' ,'Prep':'K3','Data':d2k3})

# y_  = DATA_TARGET

## Kombinasi 1

In [ ]:
# import matplotlib.pyplot as plt

# metric = []
# for xi in X1:
#     X_ = xi['Data']
#     for mod in kombinasi:
# #         xP = X_[y_[y_.values == 1].index]
# #         xN = X_[y_[y_.values == -1].index][:size]
# #         X  = np.append(xP,xN)
        
# #         yP = y_[y_.values == 1]
# #         yN = y_[y_.values == -1][:size]
# #         y  = np.append(yP,yN)
        
#         X = X_
#         y = y_
        
#         metrics,mean_fpr,mean_tpr = skorStratCV(mod['pipe'],X,y)
#         name = xi['name']+' '+xi['Prep']+' '+mod['name']
#         plt.plot(mean_fpr, mean_tpr,label='Mean AUC %s (area = %0.2f)' % (name,metrics[5]), lw=2)
        
#         metric.append({'name':name,'metrics':metrics})

# plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
# plt.xlim([-0.05, 1.05])
# plt.ylim([-0.05, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic of Kombinasi1')
# plt.legend(loc="lower right")
# plt.show()

## Kombinasi 2

In [ ]:
# import matplotlib.pyplot as plt

# metric = []
# for xi in X2:
#     X_ = xi['Data']
#     for mod in kombinasi:
# #         xP = X_[y_[y_.values == 1].index]
# #         xN = X_[y_[y_.values == -1].index][:size]
# #         X  = np.append(xP,xN)
        
# #         yP = y_[y_.values == 1]
# #         yN = y_[y_.values == -1][:size]
# #         y  = np.append(yP,yN)
        
#         X = X_
#         y = y_
        
#         metrics,mean_fpr,mean_tpr = skorStratCV(mod['pipe'],X,y)
#         name = xi['name']+' '+xi['Prep']+' '+mod['name']
#         plt.plot(mean_fpr, mean_tpr,label='Mean AUC %s (area = %0.2f)' % (name,metrics[5]), lw=2)
        
#         metric.append({'name':name,'metrics':metrics})

# plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
# plt.xlim([-0.05, 1.05])
# plt.ylim([-0.05, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic of Kombinasi1')
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
# metDum = metric

## Kombinasi 3

In [ ]:
# import matplotlib.pyplot as plt

# metric = []
# for xi in X3:
#     X_ = xi['Data']
#     for mod in kombinasi:
# #         xP = X_[y_[y_.values == 1].index]
# #         xN = X_[y_[y_.values == -1].index][:size]
# #         X  = np.append(xP,xN)
        
# #         yP = y_[y_.values == 1]
# #         yN = y_[y_.values == -1][:size]
# #         y  = np.append(yP,yN)
        
#         X = X_
#         y = y_
        
#         metrics,mean_fpr,mean_tpr = skorStratCV(mod['pipe'],X,y)
#         name = xi['name']+' '+xi['Prep']+' '+mod['name']
#         plt.plot(mean_fpr, mean_tpr,label='Mean AUC %s (area = %0.2f)' % (name,metrics[5]), lw=2)
        
#         metric.append({'name':name,'metrics':metrics})

# plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
# plt.xlim([-0.05, 1.05])
# plt.ylim([-0.05, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic of Kombinasi1')
# plt.legend(loc="lower right")
# plt.show()

In [87]:
# from ipy_table import *
# metrics_table = []
# tables = []
# columns = (['id', 'nama','time','p_1',
#             'r_1','F1 score','accuracy','AUC'])
# tables.append(columns)

# i = 0

# for me in metric:
#     i += 1
    
#     metr = []
#     metr.append(i)
#     metr.append(me['name'])
#     for m in me['metrics']:
#         metr.append(m)

#     tables.append(metr)
#     metrics_table.append(metr)
#     #print metric
    
# make_table(tables)

# # styling
# apply_theme('basic_both')
# set_column_style(11, align='center')

In [88]:
# evals = pd.DataFrame(metrics_table,columns=columns)
# evals['Vars'] = [i.split()[0] for i in evals['nama']]
# evals['Prep'] = [i.split()[1] for i in evals['nama']]
# evals['Algs'] = [i.split()[2] for i in evals['nama']]
# # evals['Time'] = evals['time']
# evals.drop('nama',axis=1,inplace=True)
# evals.drop('id',axis=1,inplace=True)

In [89]:
# evals.to_csv('Hasil3.csv')

In [90]:
# # splitter1 = ['Judul','Short','Long']
# splitter1 = ['Feed1','Feed2']
# # splitter2 = ['Kombinasi1','Kombinasi2','Kombinasi3','Kombinasi4','Kombinasi5']
# splitter2 = ['Kombinasi1','Kombinasi2','Kombinasi3']
# eval_teks = evals[evals.Prep.isin(splitter2)]
# eval_teks = pd.DataFrame([eval_teks[eval_teks.Vars == i].mean() for i in splitter1]).transpose()
# eval_teks.set_axis(1,splitter1)
# eval_teks

In [91]:
# # splitter = ['Kombinasi1','Kombinasi2','Kombinasi3','Kombinasi4','Kombinasi5']
# splitter = ['Kombinasi1','Kombinasi2','Kombinasi3']
# eval_prep = pd.DataFrame([evals[evals.Prep == i].mean() for i in splitter]).transpose()
# eval_prep.set_axis(1,splitter)
# eval_prep

In [92]:
# splitter1 = ['MNB','SVM']
# eval_teks = evals[evals.Algs.isin(splitter1)]
# eval_teks = pd.DataFrame([eval_teks[eval_teks.Algs == i].mean() for i in splitter1]).transpose()
# eval_teks.set_axis(1,splitter1)
# eval_teks

In [ ]:
mnb

In [ ]:
#knn
# knn_params = {'n_neighbors': range(1,21), 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree'],
#               'leaf_size': [15, 30, 50, 100], 'p': [1,2]}

In [ ]:
svc
# lsvm_params = {'loss':['hinge', 'squared_hinge'],'class_weight':['balanced']}

In [ ]:
#tre
# tree_param = {"criterion": ["gini", "entropy"],"class_weight" : ['balanced']}

In [ ]:
# def skorStratCV2(clf,X,y,feedTime,feedMem,crawlTime,crawlMem,nfolds=10):
#     metrics = []
    
#     trainTime,testTime = [],[]
#     precis,recals,f1s,accs,rocs = [],[],[],[],[]
    
#     mean_tpr = 0.0
#     mean_fpr = np.linspace(0, 1, 100)
#     all_tpr = []
#     X = preprocK3(X)
#     skf = cross_validation.StratifiedKFold(y, n_folds=nfolds)
#     for train_index, test_index in skf:
#         X_train,y_train = X[train_index],y[train_index]
#         X_test,y_test   = X[test_index],y[test_index]
        
#         #training time
#         start = dt.now()
#         clf.fit(X_train, y_train)
#         end = dt.now()
#         trainTime.append((end-start).microseconds)
        
#         #testing time
#         start = dt.now()
#         yhat = clf.predict(X_test)
#         end = dt.now()
#         testTime.append((end-start).microseconds)
        
#         #score
#         precis.append(precision_score(y_test, yhat, average='binary'))
#         recals.append(recall_score(y_test, yhat, average='binary'))
#         f1s.append(f1_score(y_test, yhat, average='binary'))
#         accs.append(accuracy_score(y_test, yhat))
        
#         probas_              = clf.fit(X_train, y_train).predict_proba(X_test)
#         fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
#         mean_tpr            += interp(mean_fpr, fpr, tpr)
#         mean_tpr[0]          = 0.0
        
#     def ave(lis):
#         return sum(lis)/len(lis)
    
#     testTime = ave(trainTime)+ave(testTime)
    
#     metrics.append(testTime)
#     metrics.append(feedTime)
#     metrics.append(feedMem)
#     metrics.append(crawlTime)
#     metrics.append(crawlMem)

# #     metrics.append(ave(testTime))
#     metrics.append(ave(precis))
#     metrics.append(ave(recals))
#     metrics.append(ave(f1s))
#     metrics.append(ave(accs))
    
#     mean_tpr /= nfolds
#     mean_tpr[-1] = 1.0
#     mean_auc = auc(mean_fpr, mean_tpr)
#     metrics.append(mean_auc)
    
#     return metrics,mean_fpr,mean_tpr